# Ragas `BaseModel`

> An Extended version of Pydantics `BaseModel` for some ragas specific stuff

In [1]:
#| default_exp model.pydantic_model

In [1]:
#| export
import typing as t

from pydantic import BaseModel, PrivateAttr

from ragas_annotator.typing import FieldMeta as RagasFieldMeta

In [2]:
#| export
class ExtendedPydanticBaseModel(BaseModel):
    """Extended Pydantic BaseModel with database integration capabilities"""
    
    # Private attribute for storing the database row_id
    _row_id: t.Optional[int] = PrivateAttr(default=None)
    
    # Class variable for storing column mapping overrides
    __column_mapping__: t.ClassVar[t.Dict[str, str]] = {}
    
    def __init__(self, **data):
        super().__init__(**data)
        # Initialize column mapping if not already defined
        if not self.__class__.__column_mapping__:
            self._initialize_column_mapping()
    
    @classmethod
    def _initialize_column_mapping(cls):
        """Initialize mapping from field names to column IDs."""
        for field_name, field_info in cls.model_fields.items():
            # Check if field has Column metadata (for Pydantic v2)
            column_id = None
            for extra in field_info.metadata or []:
                if isinstance(extra, RagasFieldMeta) and extra.id:
                    column_id = extra.id
                    break
            
            # If no Column metadata found, use field name as column ID
            if not column_id:
                column_id = field_name
            
            cls.__column_mapping__[field_name] = column_id
    
    @classmethod
    def get_column_id(cls, field_name: str) -> str:
        """Get the column ID for a given field name."""
        if field_name not in cls.__column_mapping__:
            raise ValueError(f"No column mapping found for field {field_name}")
        return cls.__column_mapping__[field_name]
    
    @classmethod
    def set_column_id(cls, field_name: str, column_id: str):
        """Set the column ID for a given field name."""
        if field_name not in cls.model_fields:
            raise ValueError(f"Field {field_name} not found in model")
        cls.__column_mapping__[field_name] = column_id
    
    def get_db_field_mapping(self) -> t.Dict[str, str]:
        """Get a mapping from field names to column IDs for this model."""
        return self.__class__.__column_mapping__



In [12]:
import ragas_annotator.typing as rt

In [13]:
# Example usage
class TestDataRow(ExtendedPydanticBaseModel):
    id: t.Optional[int] = None
    query: t.Annotated[str, rt.Text(id="search_query")]
    persona: t.List[t.Literal["opt1", "opt2", "opt3"]]

In [14]:
TestDataRow.__column_mapping__

{}

In [15]:
t = TestDataRow(id=1, query="this is a test", persona=["opt1"])

In [16]:
t.__column_mapping__

{'id': 'id', 'query': 'search_query', 'persona': 'persona'}